In [1]:
import dask.dataframe as dd
import polars as pl
import polars.selectors as ps
from sklearn.impute import SimpleImputer

# 1 USANDO DASK

In [2]:
df_dd = dd.read_csv(
    './dados/*.csv',
    encoding='latin',
    assume_missing=True,
    dtype={
        'RELIGIAO': 'object',
        'VINCULACAO_ANO': 'object'
    }
)

In [3]:
df_dd.head()

,ANO_NASCIMENTO,PESO,ALTURA,CABECA,CALCADO,CINTURA,RELIGIAO,MUN_NASCIMENTO,UF_NASCIMENTO,PAIS_NASCIMENTO,...,ESCOLARIDADE,VINCULACAO_ANO,DISPENSA,ZONA_RESIDENCIAL,MUN_RESIDENCIA,UF_RESIDENCIA,PAIS_RESIDENCIA,JSM,MUN_JSM,UF_JSM
0,1989.0,95.0,181.0,57.0,44.0,98.0,<NA>,DESCALVADO,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRASSUNUNGA,SP,BRASIL,5/93 - PIRASSUNUNGA,PIRASSUNUNGA,SP
1,1989.0,64.0,173.0,57.0,41.0,76.0,<NA>,CATANDUVA,SP,BRASIL,...,Ensino Superior Completo,2007,Sem dispensa,Urbana,SAO PAULO,SP,BRASIL,4/94 - SAO PAULO (BUTANTA),SAO PAULO,SP
2,1989.0,82.0,171.0,58.0,41.0,94.0,<NA>,CATANDUVA,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,CATANDUVA,SP,BRASIL,5/27 - CATANDUVA,CATANDUVA,SP
3,1989.0,NaN,NaN,NaN,NaN,NaN,<NA>,SANTO ANDRE,SP,BRASIL,...,Ensino Médio Completo,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL,6/127 - PIRAJUI,PIRAJUI,SP
4,1989.0,NaN,NaN,NaN,NaN,NaN,<NA>,PIRAJUI,SP,BRASIL,...,3ª Série do Ensino Médio,2007,Sem dispensa,Urbana,PIRAJUI,SP,BRASIL,6/127 - PIRAJUI,PIRAJUI,SP


In [4]:
df_dd.columns

Index(['ANO_NASCIMENTO', 'PESO', 'ALTURA', 'CABECA', 'CALCADO', 'CINTURA',
       'RELIGIAO', 'MUN_NASCIMENTO', 'UF_NASCIMENTO', 'PAIS_NASCIMENTO',
       'ESTADO_CIVIL', 'SEXO', 'ESCOLARIDADE', 'VINCULACAO_ANO', 'DISPENSA',
       'ZONA_RESIDENCIAL', 'MUN_RESIDENCIA', 'UF_RESIDENCIA',
       'PAIS_RESIDENCIA', 'JSM', 'MUN_JSM', 'UF_JSM'],
      dtype='object')

In [5]:
df_dd = df_dd.drop(columns={
    'ANO_NASCIMENTO',
    'CABECA', 
    'CALCADO', 
    'CINTURA',
    'RELIGIAO',
    'JSM',
    'MUN_JSM', 
    'UF_JSM'
})

In [6]:
df_dd.dtypes

PESO                        float64
ALTURA                      float64
MUN_NASCIMENTO      string[pyarrow]
UF_NASCIMENTO       string[pyarrow]
PAIS_NASCIMENTO     string[pyarrow]
ESTADO_CIVIL        string[pyarrow]
SEXO                string[pyarrow]
ESCOLARIDADE        string[pyarrow]
VINCULACAO_ANO      string[pyarrow]
DISPENSA            string[pyarrow]
ZONA_RESIDENCIAL    string[pyarrow]
MUN_RESIDENCIA      string[pyarrow]
UF_RESIDENCIA       string[pyarrow]
PAIS_RESIDENCIA     string[pyarrow]
dtype: object

In [7]:
#dd.to_parquet(
#    df_dd, 
#    './dados_parquet/', 
#    engine='pyarrow', 
#    write_index=False
#)

# 2 USANDO POLARS

In [8]:
df_pl = pl.read_parquet('./dados_parquet/')

In [9]:
df_pl.head()

PESO,ALTURA,MUN_NASCIMENTO,UF_NASCIMENTO,PAIS_NASCIMENTO,ESTADO_CIVIL,SEXO,ESCOLARIDADE,VINCULACAO_ANO,DISPENSA,ZONA_RESIDENCIAL,MUN_RESIDENCIA,UF_RESIDENCIA,PAIS_RESIDENCIA
f64,f64,str,str,str,str,str,str,str,str,str,str,str,str
95.0,181.0,"""DESCALVADO""","""SP""","""BRASIL""","""Solteiro""","""M""","""Ensino Médio Completo""","""2007""","""Sem dispensa""","""Urbana""","""PIRASSUNUNGA""","""SP""","""BRASIL"""
64.0,173.0,"""CATANDUVA""","""SP""","""BRASIL""","""Casado""","""M""","""Ensino Superior Completo""","""2007""","""Sem dispensa""","""Urbana""","""SAO PAULO""","""SP""","""BRASIL"""
82.0,171.0,"""CATANDUVA""","""SP""","""BRASIL""","""Solteiro""","""M""","""Ensino Médio Completo""","""2007""","""Sem dispensa""","""Urbana""","""CATANDUVA""","""SP""","""BRASIL"""
null,null,"""SANTO ANDRE""","""SP""","""BRASIL""","""Solteiro""","""M""","""Ensino Médio Completo""","""2007""","""Sem dispensa""","""Urbana""","""PIRAJUI""","""SP""","""BRASIL"""
null,null,"""PIRAJUI""","""SP""","""BRASIL""","""Solteiro""","""M""","""3ª Série do Ensino Médio""","""2007""","""Sem dispensa""","""Urbana""","""PIRAJUI""","""SP""","""BRASIL"""


# 2.1 MUDANDO TIPOS

In [10]:
df_pl.schema

Schema([('PESO', Float64),
        ('ALTURA', Float64),
        ('MUN_NASCIMENTO', String),
        ('UF_NASCIMENTO', String),
        ('PAIS_NASCIMENTO', String),
        ('ESTADO_CIVIL', String),
        ('SEXO', String),
        ('ESCOLARIDADE', String),
        ('VINCULACAO_ANO', String),
        ('DISPENSA', String),
        ('ZONA_RESIDENCIAL', String),
        ('MUN_RESIDENCIA', String),
        ('UF_RESIDENCIA', String),
        ('PAIS_RESIDENCIA', String)])

In [11]:
numeric_cols = df_pl.select(ps.numeric()).columns
numeric_cols

['PESO', 'ALTURA']

In [12]:
for col in numeric_cols:
    df_pl = df_pl.with_columns(
        pl.col(col).cast(pl.Int16)
    )

In [13]:
df_pl.select('VINCULACAO_ANO').unique().to_series().to_list()

['2007',
 '2017',
 '2021',
 '2014',
 'Com dispensa',
 '2010',
 '2012',
 '2009',
 '2024',
 '2023',
 '2022',
 '2020',
 '2013',
 '2016',
 '2015',
 'Sem dispensa',
 '2008',
 '2011',
 '2018',
 '2019']

In [14]:
df_pl = df_pl.filter(
    ~pl.col('VINCULACAO_ANO').is_in(['Sem dispensa', 'Com dispensa'])
)

In [15]:
df_pl.select('VINCULACAO_ANO').unique().to_series().to_list()

['2008',
 '2024',
 '2023',
 '2013',
 '2020',
 '2019',
 '2012',
 '2010',
 '2016',
 '2015',
 '2011',
 '2009',
 '2022',
 '2007',
 '2021',
 '2017',
 '2014',
 '2018']

In [16]:
df_pl = df_pl.with_columns(
    pl.col('VINCULACAO_ANO').cast(pl.Int16)
)

In [17]:
str_cols = df_pl.select(ps.by_dtype(pl.String)).columns

In [18]:
for col in str_cols:
    df_pl = df_pl.with_columns(
        pl.col(col).cast(pl.Categorical)
    )

In [19]:
df_pl.schema

Schema([('PESO', Int16),
        ('ALTURA', Int16),
        ('MUN_NASCIMENTO', Categorical),
        ('UF_NASCIMENTO', Categorical),
        ('PAIS_NASCIMENTO', Categorical),
        ('ESTADO_CIVIL', Categorical),
        ('SEXO', Categorical),
        ('ESCOLARIDADE', Categorical),
        ('VINCULACAO_ANO', Int16),
        ('DISPENSA', Categorical),
        ('ZONA_RESIDENCIAL', Categorical),
        ('MUN_RESIDENCIA', Categorical),
        ('UF_RESIDENCIA', Categorical),
        ('PAIS_RESIDENCIA', Categorical)])

## 2.2 VALORES NULOS

### 2.2.1 VALORES NUMÉRICOS

In [ ]:
for col in numeric_cols:
    tem_nulos = df_pl.select(
        pl.col(col).is_null().sum()
    ).item()
    print(f'A coluna {col} possui {tem_nulos} valores nulos.')

A coluna PESO possui 18743140 valores nulos.
A coluna ALTURA possui 18782911 valores nulos.


In [22]:
for col in numeric_cols:
    media = df_pl.select(pl.col(col).mean()).item()
    mediana = df_pl.select(pl.col(col).median()).item()
    if (media > mediana):
        print(f'Usar a média na coluna {col}.')
    else:
        print(f'Usar a mediana na coluna {col}.')

Usar a média na coluna PESO.
Usar a mediana na coluna ALTURA.


In [23]:
for medida, col in zip(['mean', 'median'], numeric_cols):
    if medida == 'mean':
        media = df_pl.select(pl.col(col).mean()).item()
        df_pl = df_pl.with_columns(
            pl.col(col).fill_null(media)
        )
    else:
        mediana = df_pl.select(pl.col(col).median()).item()
        df_pl = df_pl.with_columns(
            pl.col(col).fill_null(mediana)
        )


In [24]:
for col in numeric_cols:
    tem_nulos = df_pl.select(
        pl.col(col).is_null().sum()
    ).item()
    print(f'A coluna {col} possui {tem_nulos} valores nulos.')

A coluna PESO possui 0 valores nulos.
A coluna ALTURA possui 0 valores nulos.


### 2.2.2 VALORES TEXTUAIS

In [25]:
for col in str_cols:
    tem_nulos = df_pl.select(
        pl.col(col).is_null().sum()
    ).item()
    print(f'A coluna {col} possui {tem_nulos} valores nulos.')

A coluna MUN_NASCIMENTO possui 0 valores nulos.
A coluna UF_NASCIMENTO possui 0 valores nulos.
A coluna PAIS_NASCIMENTO possui 0 valores nulos.
A coluna ESTADO_CIVIL possui 0 valores nulos.
A coluna SEXO possui 0 valores nulos.
A coluna ESCOLARIDADE possui 0 valores nulos.
A coluna DISPENSA possui 474635 valores nulos.
A coluna ZONA_RESIDENCIAL possui 0 valores nulos.
A coluna MUN_RESIDENCIA possui 0 valores nulos.
A coluna UF_RESIDENCIA possui 0 valores nulos.
A coluna PAIS_RESIDENCIA possui 0 valores nulos.


In [26]:
df_pl = df_pl.with_columns(
    pl.col('DISPENSA').fill_null('Não informado')
)

In [27]:
for col in str_cols:
    tem_nulos = df_pl.select(
        pl.col(col).is_null().sum()
    ).item()
    print(f'A coluna {col} possui {tem_nulos} valores nulos.')

A coluna MUN_NASCIMENTO possui 0 valores nulos.
A coluna UF_NASCIMENTO possui 0 valores nulos.
A coluna PAIS_NASCIMENTO possui 0 valores nulos.
A coluna ESTADO_CIVIL possui 0 valores nulos.
A coluna SEXO possui 0 valores nulos.
A coluna ESCOLARIDADE possui 0 valores nulos.
A coluna DISPENSA possui 0 valores nulos.
A coluna ZONA_RESIDENCIAL possui 0 valores nulos.
A coluna MUN_RESIDENCIA possui 0 valores nulos.
A coluna UF_RESIDENCIA possui 0 valores nulos.
A coluna PAIS_RESIDENCIA possui 0 valores nulos.


## 2.3 PONTOS EXTREMOS